In [86]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
df_50 = pd.read_excel('上证50.xlsx')
df_shidai = pd.read_excel('中欧时代先锋.xlsx')
df_500 = pd.read_excel('中证500.xlsx')
df_yiliao = pd.read_excel('医疗etf.xlsx')
df_bandaoti = pd.read_excel('半导体.xlsx')
df_fuguo = pd.read_excel('富国天惠.xlsx')
df_guangfa = pd.read_excel('广发双擎升级.xlsx')
df_yifangda = pd.read_excel('易方达.xlsx')
df_youse = pd.read_excel('有色金属.xlsx')
df_300 = pd.read_excel('沪深300.xlsx')
df_baijiu = pd.read_excel('白酒.xlsx')

In [3]:
def pre_process(data):
    data.set_index(['日期'],drop = True,inplace = True)
    data = data.head(-2)
    data = data['收盘价(元)'].truncate(before = '2021-01-15')
    return data


In [4]:
name_list = [df_50,df_shidai,df_500,df_yiliao,df_bandaoti,df_fuguo,df_guangfa,df_yifangda,df_youse,df_300,df_baijiu]
index_list = ["50","shidai","500","yiliao","bandaoti","fuguo","guangfa","yifangda","youse","300","baijiu"]

df = pd.concat([pre_process(i) for i in name_list],axis = 1)
df.columns = ['close_50','close_shidai','close_500','close_yiliao','close_bandaiti','close_fuguo','close_guangfa','close_yifangda','close_youse','close_300','close_baijiu']
df.dropna(how='any',axis = 0,inplace= True)
df_yield = df.diff()/df.shift(1)
df_yield.dropna(how = 'any',inplace = True ,axis  = 0)
df_yield.reset_index(inplace = True,drop = True)

In [64]:
#创建收益率array和协方差矩阵
# yield_ = df_yield.mean().values
yield_ = (df.tail(1).values - df.head(1).values)/df.head(1).values
cov_ = np.cov(df_yield,rowvar= False)

In [97]:
#导入ga模块
# DNA_SIZE = 10     #设定DNA长度      
POP_SIZE = 100     #设定种群大小
DNA_SIZE = 2 
CROSS_RATE = 0.8      #设定匹配可能性大小   
MUTATION_RATE = 0.03    #基因变异的可能性大小
N_GENERATIONS = 2000      #繁衍总次数  

In [136]:
#适应性函数（在曲线中的高度）
def get_fitness(pop1,pop2): 
    
#     temp = np.zeros((pop1.shape[0],1))
    temp = []
    for i in range(len(pop1)):
        rp = pop2[i] * yield_[0][pop1[i][0]] + (1-pop2[i])*yield_[0][pop1[i][1]] - 0.05
        varp = pop2[i] **2 * cov_[pop1[i][0],pop1[i][0]] + (1- pop2[i] )**2 * cov_[pop1[i][1],pop1[i][1]] + 2* pop2[i] *(1- pop2[i] ) * cov_[pop1[i][0],pop1[i][1]]
        sharp_ratio= rp/np.sqrt(varp)
        temp.append(sharp_ratio)
    return temp

def re_fitness(temp):
    return temp -min(temp) + 1e-3

#选择留下来的人,
def select(pop1,pop2, fitness):    
    index = np.random.choice(np.arange(POP_SIZE), size=POP_SIZE, replace=True,p=fitness/fitness.sum())
    return pop1[index],pop2[index]

#父母DNA结合过程
def crossover(parent, pop):     
    if np.random.rand() < CROSS_RATE:
        i_ = np.random.randint(0, POP_SIZE, size=1)                            
#         cross_points = np.random.randint(0, 2, size=DNA_SIZE).astype(np.bool)  
#         parent[cross_points] = pop[i_, cross_points] 
        parent[0] = pop[i_[0]][1]
    return parent

#繁衍变异过程
def mutate_com(child):
    if np.random.rand() < MUTATION_RATE:
#         child[point] = 1 if child[point] == 0 else 0
        child = np.random.randint(11,size = 2)
    return child

def mutate_w(child):
    if np.random.rand() < MUTATION_RATE:
        child = np.random.random(1)
    return child

In [99]:
#随机生成100*2的种群
pop1 = np.random.randint(11, size=(POP_SIZE, DNA_SIZE))   
pop2 = np.random.random(POP_SIZE)
# pop2 = np.append(pop2.reshape(POP_SIZE,1),(1-pop2).reshape(POP_SIZE,1),axis = 1)

In [174]:
#主循环部分
for _ in tqdm(range(N_GENERATIONS)):
    
    # 遗传进化部分
    sharp_r = get_fitness(pop1,pop2)
    fitness = re_fitness(sharp_r)
    
#     print("Most fitted DNA: ", pop[np.argmax(fitness), :])
    
    pop1,pop2 = select(pop1,pop2, fitness)
    pop1_copy,pop2_copy = pop1.copy(),pop2.copy()
    
    for i,parent in enumerate(pop1):
        child = crossover(parent, pop1_copy)
        child = mutate_com(child)
#         parent = child
        pop1[i,:] = child    #下一代替换
    
    for i, parent in enumerate(pop2):
        child = mutate_w(parent)
#         parent[:] = child
        pop2[i] = child
    
print('Best_match:',pop1[np.argmax(sharp_r)],pop2[np.argmax(sharp_r)])
print('sharp_ratio:',sharp_r[np.argmax(sharp_r)])
print('买入:' ,index_list[[pop1[np.argmax(sharp_r)]][0][0]],'比重为:',pop2[np.argmax(sharp_r)],'和:',index_list[[pop1[np.argmax(sharp_r)]][0][1]],'比重为:',1-pop2[np.argmax(sharp_r)])

100%|█████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:04<00:00, 462.38it/s]

Best_match: [8 8] 0.3632772518660238
sharp_ratio: 9.928230629585647
买入: youse 比重为: 0.3632772518660238 和: youse 比重为: 0.6367227481339762
